# Comparación de Muestras en Python

Elaborar una función en Python que permita comparar dos muestras. Estas pueden ser:

- Dos muestras provenientes de distribuciones teóricas.
- Una muestra teórica y una muestra a partir de datos.
- Dos muestras provenientes de datos.

## La función debe mostrar:
1. **Funciones de densidad** $ f_1 $ y $ f_2 $.
2. **Funciones de distribución** $ F_1 $ y $ F_2 $.
3. **Gráfica PP (prob-prob)**.
4. **Gráfica QQ (quantil-quantil)**.

Además, la función debe calcular la **distancia de Kolmogorov-Smirnov (KS)** e ilustrar en las gráficas de densidad y de distribución el punto donde se alcanza esta distancia KS.

## Prueba de hipótesis de Kolmogorov-Smirnov
Se debe realizar una prueba de hipótesis de Kolmogorov-Smirnov para comparar las muestras.

## Experimentos a realizar
Usar alguno de los experimentos del ejercicio anterior (con un valor $ p $ y $ N $ fijo) y comparar la distribución obtenida del experimento contra una muestra generada aleatoriamente de la distribución geométrica:

1. $ \text{Geom}(p) $
2. $ \text{Geom}(q) $, para $ q = 1.2p $ (asegurando que $ 0 < q < 1 $).